## 트리의 앙상블

In [1]:
# 정형데이터에 가장 뛰어난 성능을 보이는 모델들
# 앙상블 모델들은 결정트리(Decision Tree)를 기반으로 만들어짐
# 앙상블 모델들
#  - 랜덤포레스트(Random Forest)
#  - 엑스트라 트리(Extra Trees)
#  - 그레디언트 부스팅(Gradient Boosting)
#  - 히스토그램 기반 그레디언트 부스팅(Histogram-base Gradient Boosting)

## 랜덤포레스트(Random Forest)

In [2]:
### 앙상블 모델 중에 가장 대표격 모델로 사용됨
# - 안정적인 성능으로 널리 사용됨
# - 앙상블 모델 중에 가장 먼저 시도하는 모델이라고 보면 됨
# - 훈련데이터에서 과대적합되는 것을 막아줌
# - 검증데이터와 데스트데이터에서 안정적인 성능을 얻을 수 있음

### 학습 개념
# - 결정트리 하나하나를 랜덤하게 만들어서 숲을 만듦
# - 훈련데이터에서 랜덤하게 샘플을 추출해 훈련을 완료한 후 다시 원본 훈련데이터에 반환
# - 랜덤하게 추출 시 이전에 사용된 샘플을 사용할 수도 있음.
#   (중복 허용)

### 부트스트랩 샘플(Bootstrap Sample)
# - 위에 설명한 랜덤한 샘플 추출 시 중복을 허용하여 데이터를 샘플링 하는 방식
# - 샘풀 추출 방식
# 1. 원본에서 랜덤 샘플 추출
# 2. 훈련 이후 사용이 끝나면 원본에 반환
# 3. 다시 원본에서 샘플 추출, 이때 중복값 추출될 수 있음
#  위 순서를 반복하면서 샘플링 통해 훈련하는 방식을 랜덤포레스트가 적용

### *** 랜덤포레스트는 교차검증을 허용함 *** ###

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('./data/08_wine.csv')

# 독립/종속변수 분리
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, random_state=42)

print(train_input.shape)
print(test_input.shape)

(4872, 3)
(1625, 3)


### 훈련모델 생성

In [2]:
# 랜덤포레스트 클래스(모델) : RandomForestClassifier()
# 교차검증 : cross_validate()
# 교차검증 후 훈련검증결과와 테스트검증결과 확인
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

# 랜덤포레스트 객체생성 : 코어 모두 사용
rf = RandomForestClassifier(n_jobs = -1, random_state=42)

# 교차검증 진행
# - return_train_score : 검증결과 반환받기
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs = -1)

# 최종 훈련평가 결과 및 검증결과
print(scores)
print('=============')
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

{'fit_time': array([0.23727798, 0.26207328, 0.24305344, 0.25969148, 0.28168511]), 'score_time': array([0.05032182, 0.03377819, 0.03663635, 0.03333354, 0.03539014]), 'test_score': array([0.88      , 0.90051282, 0.90349076, 0.89014374, 0.88295688]), 'train_score': array([0.99743392, 0.99692071, 0.99846075, 0.99820421, 0.99820421])}
0.997844759088341 0.8914208392565683


In [5]:
rf.fit(train_input, train_target)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [6]:
# 특성중요도
print(rf.feature_importances_)

[0.23155241 0.49706658 0.27138101]


In [7]:
### oob 기능사용
# 훈련에 참여하지 못한 잔여샘플 사용하는 기능
# 기본은 사용한함
rf = RandomForestClassifier(oob_score = True, n_jobs=-1,random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8981937602627258


## 엑스트라 트리(Extra Tree)

In [8]:
# - 랜덤포레스트와 유사하게 작동
# - 기본적으로 100개의 결정트리를 훈련
# - 랜덤포레스트와의 차이점
#   : 부트스트램 샘플링을 지원하지 않음
#   : 훈련데이터 전체를 이용하여 결정트리를 생성
#   : 무작위로 트리를 분리함
# - 사용되는 속성 : splitter = 'random' 무작위속성
# - 장점
#   : 과대적합을 막고, 검증데이터의 평가 값을 높일 수 있음
#   : 특성 데이터가 많지 않은 경우에는 랜덤포레스트와 큰 차이가 없음
# - 랜덤포레스트는 불순도 등 여러가지 조건에 따라 결정트리를 생성하기 때문에 속도가 느린 반면
# - 엑스트라트리는 랜덤하게 결정트리를 생성하기에 속도가 다소 빠르다는 장점

In [9]:
### 사용패키지 : 랜덤포레스트와 동일
# 사용되는 클래스(모델) : ExtraTreesClassifier()

### 코어 전체사용, train 및 test 결과값 출력
### 최종 train 및 test 결과 확인

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_validate

et =  ExtraTreesClassifier(n_jobs = -1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs = -1)

# 최종 훈련평가 결과 및 검증결과
print(scores)
print('=============')
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

{'fit_time': array([0.1497252 , 0.15935373, 0.15935373, 0.17240953, 0.17241263]), 'score_time': array([0.03480339, 0.03099108, 0.03099108, 0.02692723, 0.02692676]), 'test_score': array([0.89128205, 0.89128205, 0.89938398, 0.88706366, 0.88295688]), 'train_score': array([0.99743392, 0.99692071, 0.99846075, 0.99820421, 0.99820421])}
0.997844759088341 0.8903937240035804


In [10]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20702369 0.51313261 0.2798437 ]


## 그레디언트 부스팅(Gradient Boosting)

In [11]:
### 깊이(max_depth)가 얕은 결정트리를 사용함
# - 기본적으로 max_depth = 3 사용
# - 결정트리는 100개 사용
### *** 기존에 다른 훈련모델의 결과가 좋지 않을 때 사용하는 모델 *** ###
# 기존 훈련모델의 오차를 많이 보완해줌
# 성능 향상을 위한 모델로 주로 사용
# 과대적합에 강하고 일반화(과대/과소적합이 없는 상태)에 강함

# 성능향상 테스트 방법
# - 결정트리의 갯수를 조절하면서 테스트 진행
# - 학습률을 지원하기 때문에 학습률의 값을 증가시키면서 테스트 진행
#   : 기본 학습률은 0.1

# 단점
# - 순서대로 트리를 추가(랜덤x)하지 않기 때문에
# - 훈련 속도는 느림
# - 이런 느린 속도를 개선한 모델이
#   히스토그램 기반 그레디언트 부스팅 모델

### 그레디언트 부스팅 모델 생성

In [12]:
### 사용하는 클래스(모델) : GradientBoostingClassifier
# 객체 생성시 아무것도 안주고 seed값만 
# 교차검증시에는 train, test 결과값 출력
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs= -1)

print(scores)
print('=============')
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

{'fit_time': array([0.27227759, 0.2742753 , 0.28724003, 0.28823686, 0.28303766]), 'score_time': array([0.00299215, 0.00215602, 0.00199461, 0.00235224, 0.00199413]), 'test_score': array([0.86461538, 0.87794872, 0.88090349, 0.8613963 , 0.87268994]), 'train_score': array([0.89299461, 0.88555299, 0.88660852, 0.89276552, 0.88943048])}
0.8894704231708938 0.8715107671247301


In [13]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)
print(gb.score(test_input, test_target))

[0.12517641 0.73300095 0.14182264]
0.8578461538461538


### 학습률 적용하기

In [14]:
# : 학습률이 커지면 트리 보정을 가하게 하기 때문에
#   복잡한 모델을 만들어서 일반화 성능을 떨어트리게 됨
# 학습률 : learning_rate = 0.1 -> 기본값

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate

gb = GradientBoostingClassifier(n_estimators = 500,
                                learning_rate = 0.2,
                                random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs= -1)

print(scores)
print('=============')
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

{'fit_time': array([1.23122787, 1.32029867, 1.36013174, 1.30635643, 1.28100109]), 'score_time': array([0.00598407, 0.01008248, 0.        , 0.00154972, 0.00614977]), 'test_score': array([0.88      , 0.88512821, 0.89527721, 0.862423  , 0.87577002]), 'train_score': array([0.95150115, 0.94842186, 0.94946126, 0.95177014, 0.95484864])}
0.9512006117505237 0.879719686200179


## 히스토그램기반 그레디언트 부스팅(Histogram-base Gradient Boosting)

In [15]:
## 사용하는 클래스(모델) : HistGradientBoostingClassifier()
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_validate

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs= -1)

print(scores)
print('=============')
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

{'fit_time': array([0.36045742, 0.33196592, 0.34485745, 0.38096833, 0.34948516]), 'score_time': array([0.01077628, 0.01095605, 0.01260805, 0.01201534, 0.01196885]), 'test_score': array([0.87179487, 0.89333333, 0.8973306 , 0.85934292, 0.88090349]), 'train_score': array([0.93815756, 0.93482166, 0.9374038 , 0.93945613, 0.94022576])}
0.9380129799494501 0.8805410414363187


In [16]:
hgb.fit(train_input, train_target)
print(hgb.score(test_input, test_target))

0.8584615384615385


## 사이킷런 이외의 다른 패키지에서 지원하는
## 히스토그램기반 그레디언트 부스팅 기능 모델들

### XGboost

In [3]:
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target,
                        return_train_score=True, n_jobs= -1)

print(scores)
print('=============')
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

### LightGBM

In [ ]:
# - 마이크로소프트에서 만든 히스토그램 기반 그레디언트 부스트 패키지
# - 훈련 속도가 매우 빠름
# - 최신 기술을 많이 적용하고 잇어서 인기가 올라가는 중

In [2]:
from lightgbm import LGBMClassifier

In [ ]:
lgb = LGBMClassfier(tree_method='hist', random_state=42)
scores = cross_validate(lgb, train_input, train_target,
                        return_train_score=True, n_jobs= -1)

print(scores)
print('=============')
print(np.mean(scores['train_score']), np.mean(scores['test_score']))